# Predicting machine failure

Predicting machine failure through machine learning is paramount for efficient industrial operations. By analysing data patterns, ML models can forecast potential breakdowns, allowing for proactive maintenance and minimising downtime. This predictive approach not only enhances equipment reliability but also optimises resource allocation.

## Scope

## Model performance metrics

## Import dependencies

In [58]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns

import sys
sys.path.insert(0, "C:\\Users\\billy\\OneDrive\\Documents\\Python Scripts\\1. Portfolio\\machine-failure\\machine-failure")
import custom_funcs as cf

## Import raw data

* **UID:** Unique identifier ranging from 1 to 10000

* **productID:** Consisting of a letter L, M, or H for low (50% of all products), medium (30%), and high (20%) as product quality variants and a variant-specific serial number

* **air temperature [K]:** Generated using a random walk process later normalized to a standard deviation of 2 K around 300 K

* **process temperature [K]:** Generated using a random walk process normalized to a standard deviation of 1 K, added to the air temperature plus 10 K.

* **rotational speed [rpm]:** Calculated from powepower of 2860 W, overlaid with a normally distributed noise

* **torque [Nm]:** Torque values are normally distributed around 40 Nm with an Ïƒ = 10 Nm and no negative values.

* **tool wear [min]:** The quality variants H/M/L add 5/3/2 minutes of tool wear to the used tool in the process. and a 'machine failure' label that indicates, whether the machine has failed in this particular data point for any of the following failure modes are true.

* **Target:** Failure or Not

* **Failure Type:** Type of Failure

In [59]:
raw_df = pd.read_csv(cf.file_directory("raw") + "predictive_maintenance.csv")
raw_df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure
